In [1]:
from flask import Flask , request , jsonify
import pyodbc
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
import threading
import time
from datetime import datetime
import requests
import json

In [2]:
app = Flask(__name__)

In [3]:
def connect_db():
    conn_str = (
        'DRIVER={ODBC Driver 17 for SQL Server};'
        'SERVER=localhost\\SQLEXPRESS;'
        'DATABASE=Projects;'
        'Trusted_Connection=yes;'
    )
    return pyodbc.connect(conn_str)

In [ ]:
@app.route('/customers' , methods=['POST'])
def insert_customers():
    data=request.json
    try:
        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute("""INSERT INTO Customers ( Name  , Gender , BirthDate , Phone , Email , City) 
                        VALUES( ? , ? , ? , ? , ? , ?)
                       """,data['Name']  ,data['Gender'], data['BirthDate'] , data['Phone'], data['Email'] , data['City'] 
                       
                       )
        conn.commit() 
        return jsonify({"message" : "customer Added"}), 201
    except Exception as e:
        return jsonify({'error': str(e)}), 400
    finally:
        conn.close()


@app.route('/branches' , methods=['POST'])
def insert_Branches():
    data=request.json
    try:
        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute("""INSERT INTO Branches ( BranchName  , City , Address)
                       VALUES (? , ? , ? )
                       """, data['BranchName']  ,data['City'], data['Address'] 
                       
                       )
        conn.commit() 
        return jsonify({"message" : "Branches Added"}), 201
    except Exception as e:
        return jsonify({'error': str(e)}), 400
    finally:
        conn.close()


             
@app.route('/products', methods=['POST'])
def insert_product():
    data = request.get_json()
    try:
        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO Products (ProductName, Category, Price, Supplier)
            VALUES (?, ?, ?, ?)
        """, (
            data['ProductName'],
            data.get('Category'),
            data.get('Price'),
            data.get('Supplier')
        ))
        conn.commit()
        cursor.close()
        conn.close()
        return jsonify({"message": "Product inserted successfully"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500.
    


@app.route('/sales' , methods=['POST'])
def insert_sales():
    data=request.json
    try:
        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute("""INSERT INTO Sales  ( CustomerID  , BranchID , SaleDate , TotalAmount)
                       VALUES (? , ? , ? , ? )
                       """, data['CustomerID']  ,data['BranchID'], data['SaleDate']  , data['TotalAmount']
                       
                       )
        conn.commit() 
        return jsonify({"message" : "Sales Added"}), 201
    except Exception as e:
        return jsonify({'error': str(e)}), 400
    finally:
        conn.close()

@app.route('/sale_details', methods=['POST'])
def insert_sale_detail():
    data = request.get_json()
    try:
        conn = connect_db()
        cursor = conn.cursor()

        cursor.execute("""
            INSERT INTO SaleDetails (SaleID, ProductID, Quantity, UnitPrice, TotalPrice)
            VALUES (?, ?, ?, ?, ?)
        """, (
            data['SaleID'],
            data['ProductID'],
            data['Quantity'],
            data['UnitPrice'],
            data['TotalPrice']
        ))

        conn.commit()

        cursor.close()
        conn.close()
        return jsonify({"message": "Sale detail inserted successfully"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


def run_flask():
    app.run(port=5000 , debug=False , use_reloader=False)

# code to run the flask API in a separate thread in the background
threading.Thread(target=run_flask).start()
time.sleep(1)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [15/Aug/2025 20:40:47] "POST /products HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 20:40:47] "POST /products HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 20:40:47] "POST /products HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 20:40:47] "POST /products HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 20:40:47] "POST /products HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 20:40:47] "POST /products HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 20:40:47] "POST /products HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 20:40:47] "POST /products HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 20:40:48] "POST /products HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 20:40:48] "POST /products HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 20:40:48] "POST /products HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 20:40:48] "POST /products HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 20:40:48] "POST /products HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 20:40:48] "POST /products HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 20:40:4

In [ ]:
endpoints = {
    # "jsonfile/customers.json": "http://127.0.0.1:5000/customers",
    # "jsonfile/branches.json": "http://127.0.0.1:5000/branches",
    "jsonfile/products.json": "http://127.0.0.1:5000/products",
    # "jsonfile/sales.json": "http://127.0.0.1:5000/sales",
    "jsonfile/sale_details.json": "http://127.0.0.1:5000/sale_details"
}

# Send each JSON file to its corresponding API
for file_name, url in endpoints.items():
    try:
        with open(file_name, "r", encoding="utf-8") as f:
            data_list = json.load(f)  # Must be a list of dicts

        for record in data_list:
            res = requests.post(url, json=record)
            print(f"POST to {url} -> {res.status_code} {res.text}")

    except FileNotFoundError:
        print(f"❌ File {file_name} not found!")
    except json.JSONDecodeError:
        print(f"❌ Invalid JSON format in {file_name}")


POST to http://127.0.0.1:5000/products -> 200 {"message":"Product inserted successfully"}

POST to http://127.0.0.1:5000/products -> 200 {"message":"Product inserted successfully"}

POST to http://127.0.0.1:5000/products -> 200 {"message":"Product inserted successfully"}

POST to http://127.0.0.1:5000/products -> 200 {"message":"Product inserted successfully"}

POST to http://127.0.0.1:5000/products -> 200 {"message":"Product inserted successfully"}

POST to http://127.0.0.1:5000/products -> 200 {"message":"Product inserted successfully"}

POST to http://127.0.0.1:5000/products -> 200 {"message":"Product inserted successfully"}

POST to http://127.0.0.1:5000/products -> 200 {"message":"Product inserted successfully"}

POST to http://127.0.0.1:5000/products -> 200 {"message":"Product inserted successfully"}

POST to http://127.0.0.1:5000/products -> 200 {"message":"Product inserted successfully"}

POST to http://127.0.0.1:5000/products -> 200 {"message":"Product inserted successfully"}
